In [1]:
import json
import pandas as pd
import glob
from pandas import json_normalize

In [2]:
files = glob.glob('json_transcripts/unlimited/*.json' ,recursive=True)

#this is from the season of Critical Role she hosted so the process & format is identical to cr_df

In [3]:
exudict = {}
def GetFile(filename):
    for file in files:
        results = file.split('-')  #split file path at '-'
        epnum = results[1]         #index to 1 to get episodenum.json
        epnum = epnum[:-5]         #index to get rid of '.json'
        f = open(file, 'r')        #read in files
        text = f.read()
        f.close
        exudict[epnum]=text

In [4]:
GetFile(files)

In [5]:
len(exudict) #a short season!

8

In [6]:
exulist = []
def buildDF(feddict):         #feed dict created by GetFile function
    for file in feddict:        #for each file in dict
        info = json.loads(feddict[str(file)])     #load str of value from fed dict
        base_df = json_normalize(info)            #first level normalize
        lines_df = json_normalize(base_df['lines'])   #second level normalize
        lines_long = pd.melt(lines_df)                #melt columns into rows
        ts_df = json_normalize(lines_long['value'])   #third level normalize
        ts_df['episode']=file                         # add column to final df populated w dict key
        exulist.append(ts_df)

In [7]:
buildDF(exudict)

In [8]:
len(exulist)

8

In [9]:
exu_df = pd.concat(exulist)

In [10]:
exu_df.head()

,name,text,ts.h,ts.m,ts.s,episode
0,AABRIA,Let's start with Robbie.,0,0,0,1
1,ROBBIE,I'm so excited!,0,0,18,1
2,AABRIA,Yeah!,0,0,24,1
3,ROBBIE,Whew!,0,0,26,1
4,AABRIA,Liam!,0,0,26,1


In [11]:
metalist = []
for x in exudict:           
    fullcols = json.loads(exudict[str(x)])      
    newdf = json_normalize(fullcols)
    metalist.append(newdf)

In [12]:
meta_df = pd.concat(metalist)

In [13]:
meta_df.rename(columns={'metadata.episode':'episode', 'metadata.title':'title'}, inplace=True)

In [14]:
meta_df = meta_df.iloc[:, 2:4]
meta_df.head()

,episode,title
0,1,The Nameless Ones
0,2,The Oh No Plateau
0,3,A Glorious Return
0,4,By the Road
0,5,A Test of Worth


In [15]:
exu_df = exu_df.merge(meta_df, how='left', on='episode', suffixes=('', '_y'))
exu_df.drop(exu_df.filter(regex='_y$').columns, axis=1, inplace=True)

In [16]:
exu_df.head() #I included the episode name but will likely eliminate it

,name,text,ts.h,ts.m,ts.s,episode,title
0,AABRIA,Let's start with Robbie.,0,0,0,1,The Nameless Ones
1,ROBBIE,I'm so excited!,0,0,18,1,The Nameless Ones
2,AABRIA,Yeah!,0,0,24,1,The Nameless Ones
3,ROBBIE,Whew!,0,0,26,1,The Nameless Ones
4,AABRIA,Liam!,0,0,26,1,The Nameless Ones


In [17]:
import pickle

#### D20 Data

So the D20 data is different entirely from the CR data - makes sense, different groups use different transcription criteria and norms. Here's a test file I used to check out what the data looks like and how we can get it into a suitable DF. 

Data as .json was secured by saving transcripts as html files and converting to json online. from there it was simple enough to normalize and create something usable. 

In [20]:
j = open('D20/BurrowsEnd/Episode1TheRedWarren.json', 'rb')
test = j.read()
j.close()

In [87]:
test[1300:1500] #a sneak peek into what the data looks like raw, not nearly as nice as the CR data

b'al Recall!": null, "Erika:\\u00a0What\'s she gonna call us? What\'s she gonna call us?": null, "Jasper: [sing-song] I don\'t know what she\'s gonna call us!": null, "Brennan:\\u00a0I don\'t know!": null, "Aa'

In [22]:
data = json.loads(test)      #read in isolated ep1
df = json_normalize(data)
df.head()

,Dimension 20 Season 20,Burrow’s End,Starring: Aabria Iyengar as Game Master,Brennan Lee Mulligan as Tula; Erika Ishii as Ava;,Siobhan Thompson as Jaysohn; Rashawn Nadine Scott as Viola;,"Isabella ""Izzy"" Roland as Lila; and Jasper William Cartwright as Thorn",Vale,Episode 1: The Red Warren,[Next Episode] >,"The Dimension 20 logo appears on screen, then is covered in darkness in",...,Thorn: I was entirely correct to be terrified of your family. Wow.,Aabria: Bring out the map.,Jasper: Oh no!,[all screaming],Siobhan: So this is why you emailed us a trigger warning.,"Aabria: A little bit, yeah.",[melancholy piano music],------------------------------------------------------------------------,Captions extracted by: Rachel S.,Edited by: Rachel S.
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [23]:
test_long = pd.melt(df) 
test_long

#It's not perfect, but with our old buddy regex we can get it into working order

,variable,value
0,Dimension 20 Season 20,None
1,Burrow’s End,None
2,Starring: Aabria Iyengar as Game Master,None
3,Brennan Lee Mulligan as Tula; Erika Ishii as Ava;,None
4,Siobhan Thompson as Jaysohn; Rashawn Nadine Sc...,None
...,...,...
2112,"Aabria: A little bit, yeah.",None
2113,[melancholy piano music],None
2114,----------------------------------------------...,None
2115,Captions extracted by: Rachel S.,None


In [25]:
test_long[300:315]

,variable,value
300,Projection on the dome walls transitions into ...,None
301,Jasper: What?,None
302,[Jasper saying escalating 'What's],None
303,Projection on the dome reveals a shadow-play s...,None
304,intro sequence. The players gape with excitement.,None
305,"Erika: [shouting] No! No, you didn't! You didn't!",None
306,[tranquil music],None
307,Beatrix: Everything begins with the Blue. Ther...,None
308,"and the blue of the water, and the Blue of the...",None
309,The shadow-play screen shows a family of stoat...,None


Some key things I'm noticing with our exploratory look is that it looks like ALL audio is in square brackets, which is great. That makes that easier to split out into a "nonspeech column". 

This data uses speaker names and character names both, which is interesting and valuable, but not entirely to our needs. However I will be able to use this to create a name column, and since I know who plays which character thanks to the metadata (and from watching) I'll make a secondary column for "speaker" associated with whichever character name.

The transcripts here includes non-audio information about the room and actions at the table like "The shadow-play screen shows a family of stoat(s)" or "Erika can barely contain themselves." but it isn't uniquely marked as room information or otherwise nonspeech. This will likely be the biggest challenge in getting separated from actual speech, but I'll explore more once we have a full df

#### Reading in all the D20 Data

In [26]:
files = glob.glob('D20/ACOFF/*.json' ,recursive=True)

In [27]:
files

['D20/ACOFF\\Episode10YouWillNeverKnowaLonelyDayAgain.json',
 'D20/ACOFF\\Episode1ABloomtoRemember.json',
 'D20/ACOFF\\Episode2TheGreatHartHunt.json',
 'D20/ACOFF\\Episode3DuelontheSouthernLawn.json',
 'D20/ACOFF\\Episode4SeasideTeatime.json',
 'D20/ACOFF\\Episode5ThroughtheHedgeMaze.json',
 'D20/ACOFF\\Episode6OfOneMind.json',
 'D20/ACOFF\\Episode7TheMasqueradeBall.json',
 'D20/ACOFF\\Episode8TheRuleofSneakery.json',
 'D20/ACOFF\\Episode9TheaterofWar.json']

In [28]:
teststr = 'D20/ACOFF\\Episode10YouWillNeverKnowaLonelyDayAgain.json'

In [29]:
teststr = teststr.split('\\')

In [30]:
teststr[1][7:9]

#this won't be perfect, it'll overcapture in the case of single digit episodes but we'll fix that later

'10'

I created amended versions of the CR functions I used to read in the files and make them into dataframes. The file names are different so the splitting looks different too, and it only needs one level of normalization followed by melting

In [31]:
D20dict = {}
def GetFile(filename):
    for file in files:
        results = file.split('\\')  #split file path at '\\'
        epnum = results[1][7:9]      #index into 1 to get episodenum
        f = open(file, 'r')        #read in files
        text = f.read()
        f.close
        D20dict[epnum]=text     #add to dict with episode num as key & text as value

In [32]:
GetFile(files)

In [33]:
len(D20dict)

10

In [34]:
D20list = []
def buildDF(feddict):         #feed dict created by GetFile function
    for file in feddict:        #for each file in dict
        info = json.loads(feddict[str(file)])     #load str of value from fed dict
        base_df = json_normalize(info)            #first level normalize
        new_df = pd.melt(base_df)                #melt columns into rows
        new_df['episode']=file                         # add column to final df populated w dict key
        D20list.append(new_df)

In [35]:
buildDF(D20dict)

In [36]:
len(D20list)  #keep track of episode number files - sanity check

10

In [37]:
ACOFF_df = pd.concat(D20list)

In [38]:
ACOFF_df.head()   #looks good, looks treatable

,variable,value,episode
0,Dimension 20 Season 14,None,10
1,A Court of Fey & Flowers,None,10
2,Starring: Aabria Iyengar as Game Master,None,10
3,Surena Marie as BINX Choppley; Oscar Montoya a...,None,10
4,Lou Wilson as Lord Squak Airavis; Emily Axford...,None,10


In [39]:
ACOFF_df.episode.value_counts()  #this is the single digit issue I predicted, but it's cleanable as well

3D    2913
7T    2855
2T    2842
8T    2542
5T    2430
6O    2368
9T    1991
1A    1896
4S    1871
10    1318
Name: episode, dtype: int64

In [88]:
#since the D20 data is made up of a bunch of mini seasons, I wanted to keep this info present
#A Court of Fey & Flowers 

ACOFF_df['season'] = 'ACOFF'   
ACOFF_df.head()

,variable,value,episode,season
0,Dimension 20 Season 14,None,10,ACOFF
1,A Court of Fey & Flowers,None,10,ACOFF
2,Starring: Aabria Iyengar as Game Master,None,10,ACOFF
3,Surena Marie as BINX Choppley; Oscar Montoya a...,None,10,ACOFF
4,Lou Wilson as Lord Squak Airavis; Emily Axford...,None,10,ACOFF


In [41]:
D20list.clear()
D20dict.clear()
print(len(D20list))
print(len(D20dict))

#clear the dict and list populated so we can reuse the function freshly

0
0


In [42]:
#rinse and repeat
#A Court of Fey & Flowers Adventuring Party (after episode chat show)

files = glob.glob('D20/ACOFF_AP/*.json' ,recursive=True)
files

['D20/ACOFF_AP\\Episode10SchrdingersCanon.json',
 'D20/ACOFF_AP\\Episode11TheWishFactory.json',
 'D20/ACOFF_AP\\Episode1IDemandSatisfaction.json',
 'D20/ACOFF_AP\\Episode2ADiamondDoesntKnowTheyreaDiamond.json',
 'D20/ACOFF_AP\\Episode3WasYourFatheraMeatBurglar.json',
 'D20/ACOFF_AP\\Episode4NotTooBadForaKidWhoUsedtoLickWalls.json',
 'D20/ACOFF_AP\\Episode5TrinketsForEverybody.json',
 'D20/ACOFF_AP\\Episode6KnickolasPterodactylHob.json',
 'D20/ACOFF_AP\\Episode7ASpillofPearls.json',
 'D20/ACOFF_AP\\Episode8DoctorGreenHunter.json',
 'D20/ACOFF_AP\\Episode9DoClothesDie.json']

In [44]:
GetFile(files)
len(D20dict)

11

In [45]:
buildDF(D20dict)
len(D20list)

11

In [46]:
ACOFFAP_df = pd.concat(D20list)
ACOFFAP_df['season'] = 'ACOFF AP'
ACOFFAP_df.head()

,variable,value,episode,season
0,Dimension 20,None,10,ACOFF AP
1,Adventuring Party,None,10,ACOFF AP
2,,None,10,ACOFF AP
3,Schrödinger’s Canon,None,10,ACOFF AP
4,Season 10 Episode 10,None,10,ACOFF AP


In [47]:
D20list.clear()
D20dict.clear()
print(len(D20list))
print(len(D20dict))

0
0


In [48]:
#Burrow's End 

files = glob.glob('D20/BurrowsEnd/*.json' ,recursive=True)
files

['D20/BurrowsEnd\\Episode10EvolutionandRevolution.json',
 'D20/BurrowsEnd\\Episode1TheRedWarren.json',
 'D20/BurrowsEnd\\Episode2BearingtheScars.json',
 'D20/BurrowsEnd\\Episode3ASecondSun.json',
 'D20/BurrowsEnd\\Episode4LastBast.json',
 'D20/BurrowsEnd\\Episode5ProtecttheLight.json',
 'D20/BurrowsEnd\\Episode6ReactorCharlie.json',
 'D20/BurrowsEnd\\Episode7TheFirstStoats.json',
 'D20/BurrowsEnd\\Episode8Five.json',
 'D20/BurrowsEnd\\Episode9Human.json']

In [49]:
GetFile(files)
len(D20dict)

10

In [50]:
buildDF(D20dict)
len(D20list)

10

In [51]:
burrow_df = pd.concat(D20list)
burrow_df['season'] = "Burrow's End"
burrow_df.head()

,variable,value,episode,season
0,Dimension 20 Season 20,None,10,Burrow's End
1,Burrow’s End,None,10,Burrow's End
2,StarrinAva: Aabria Iyengar as Game Master,None,10,Burrow's End
3,Brennan Lee Mulligan as Tula; Erika Ishii as Ava;,None,10,Burrow's End
4,Siobhan Thompson as Jaysohn; Rashawn Nadine Sc...,None,10,Burrow's End


In [56]:
D20list.clear()
D20dict.clear()
print(len(D20list))
print(len(D20dict))

0
0


In [57]:
#Burrow's End Adventuring Party

files = glob.glob('D20/BurrowsEnd_AP/*.json' ,recursive=True)
files

['D20/BurrowsEnd_AP\\Episode1StoatalRecall.json',
 'D20/BurrowsEnd_AP\\Episode2DoesaBearGetEatenOutintheWoods.json',
 'D20/BurrowsEnd_AP\\Episode3ImHereandImJaysohn.json',
 'D20/BurrowsEnd_AP\\Episode4TheBastofUs.json',
 'D20/BurrowsEnd_AP\\Episode5BigEmotionsAreSoFun.json']

In [58]:
GetFile(files)
len(D20dict)

5

In [59]:
buildDF(D20dict)
len(D20list)

5

In [60]:
burrowAP_df = pd.concat(D20list)
burrowAP_df['season'] = "Burrow's End AP"
burrowAP_df.head()

,variable,value,episode,season
0,Dimension 20,None,1S,Burrow's End AP
1,Adventuring Party,None,1S,Burrow's End AP
2,,None,1S,Burrow's End AP
3,Stoatal Recall,None,1S,Burrow's End AP
4,Season 15 Episode 1,None,1S,Burrow's End AP


In [62]:
D20list.clear()
D20dict.clear()
print(len(D20list))
print(len(D20dict))

0
0


In [63]:
#Misfits & Magic (this is complete data, it's a short series)

files = glob.glob('D20/Misfits_and_Magic/*.json' ,recursive=True)
files

['D20/Misfits_and_Magic\\Episode1TheChosenOnes.json',
 'D20/Misfits_and_Magic\\Episode2ClassConflict.json',
 'D20/Misfits_and_Magic\\Episode3FamilyOnSix.json',
 'D20/Misfits_and_Magic\\Episode4WeretheHeroes.json',
 'D20/Misfits_and_Magic\\MisfitsMagicHolidaySpecial.json']

In [64]:
GetFile(files)
len(D20dict)

5

In [65]:
buildDF(D20dict)
len(D20list)

5

In [66]:
mismag_df = pd.concat(D20list)
mismag_df['season'] = "Misfits and Magic"
mismag_df.head()

,variable,value,episode,season
0,Dimension 20,None,1T,Misfits and Magic
1,Misfits and Magic,None,1T,Misfits and Magic
2,Starring: Aabria Iyengar as Game Master,None,1T,Misfits and Magic
3,Erika Ishii as Dream;,None,1T,Misfits and Magic
4,Danielle Radford as Sam Black;,None,1T,Misfits and Magic


In [68]:
D20list.clear()
D20dict.clear()
print(len(D20list))
print(len(D20dict))

0
0


In [69]:
#Misfits & Magic Adventuring party (full)

files = glob.glob('D20/MisMag_AP/*.json' ,recursive=True)
files

['D20/MisMag_AP\\Episode1CluckifYouBuck.json',
 'D20/MisMag_AP\\Episode2TrueFactsAboutMcDonaldsApplePie.json',
 'D20/MisMag_AP\\Episode3WetFire_andBadNewsBrooms_.json',
 'D20/MisMag_AP\\Episode4ThatstheSquad.json',
 'D20/MisMag_AP\\Episode5EverybodyPutTheirLittleStankOnIt.json']

In [70]:
GetFile(files)
len(D20dict)

5

In [71]:
buildDF(D20dict)
len(D20list)

5

In [72]:
mismagAP_df = pd.concat(D20list)
mismagAP_df['season'] = "Misfits and Magic AP"
mismagAP_df.head()

,variable,value,episode,season
0,Dimension 20,None,1C,Misfits and Magic AP
1,Adventuring Party,None,1C,Misfits and Magic AP
2,Cluck if You Buck,None,1C,Misfits and Magic AP
3,Season 5 Episode 1,None,1C,Misfits and Magic AP
4,[Next Episode] >,None,1C,Misfits and Magic AP


In [74]:
D20list.clear()
D20dict.clear()
print(len(D20list))
print(len(D20dict))

0
0


In [75]:
#Misfits & Magic 2 (INCOMPLETE DATA)

files = glob.glob('D20/MisMag2/*.json' ,recursive=True)
files

['D20/MisMag2\\Episode1AMeetingofMisfits.json',
 'D20/MisMag2\\Episode2MagmaandMingle.json']

In [76]:
GetFile(files)
len(D20dict)

2

In [77]:
buildDF(D20dict)
len(D20list)

2

In [78]:
mismag2_df = pd.concat(D20list)
mismag2_df['season'] = "Misfits and Magic 2"
mismag2_df.head()

,variable,value,episode,season
0,Dimension 20 Season 23,None,1A,Misfits and Magic 2
1,Misfits and Magic Season 2,None,1A,Misfits and Magic 2
2,Starring: Aabria Iyengar as Game Master;,None,1A,Misfits and Magic 2
3,Erika Ishii as K; Danielle Radford as Sam Black;,None,1A,Misfits and Magic 2
4,Lou Wilson as Jammer Jammer; and Brennan Lee M...,None,1A,Misfits and Magic 2


In [89]:
#There was no Misfits & Magic 2 AP data YET